In [1]:
# Extra Installs for this notebook
!pip install seaborn
!pip install ipympl   
!pip install "anywidget>=0.9.0" "plotly>=5.18.0" "ipywidgets>=7.6"

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
DEPRECATION: p

In [2]:
%matplotlib widget

import ipywidgets as widgets
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Some plotting defaults for better readability
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 8)})
print("Libraries imported and interactive plotting enabled.")

Libraries imported and interactive plotting enabled.


In [3]:
import os
import sys

current_notebook_dir = os.getcwd()
ROOT_DIR = os.path.abspath(os.path.join(current_notebook_dir, '..', '..'))

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

In [4]:
print(f"Current Working Directory (Notebook location): {current_notebook_dir}")
print(f"Set Project Root Directory to: {ROOT_DIR}")
print(f"sys.path updated with ROOT_DIR.")

Current Working Directory (Notebook location): /home/lindert/TUM/ADL4CV/SemanticHyperDiffusion/scripts/act_analysis
Set Project Root Directory to: /home/lindert/TUM/ADL4CV/SemanticHyperDiffusion
sys.path updated with ROOT_DIR.


In [5]:
ROOT_DIR

'/home/lindert/TUM/ADL4CV/SemanticHyperDiffusion'

In [6]:
activations_file = 'activations/1a04e3eab45ca15dd86060f189eb133.pt'
activations_file = os.path.join(ROOT_DIR, activations_file)
if not os.path.exists(activations_file):
    print(f"Error: Activation file '{activations_file}' not found.")
    print("Please run the 'analyze_activations.py' script first.")
else:
    print(f"Loading activations from '{activations_file}'...")
    activations = torch.load(activations_file)
    print("Activations loaded successfully!")

# Let's inspect what we've loaded
print("\nAvailable layers and their activation tensor shapes:")
for name, tensor in activations.items():
    print(f"  - {name}: {tensor.shape}")

Loading activations from '/home/lindert/TUM/ADL4CV/SemanticHyperDiffusion/activations/1a04e3eab45ca15dd86060f189eb133.pt'...
Activations loaded successfully!

Available layers and their activation tensor shapes:
  - hidden_layer_0: torch.Size([2048, 128])
  - hidden_layer_1: torch.Size([2048, 128])
  - hidden_layer_2: torch.Size([2048, 128])


In [7]:
# Load the point cloud data for context
pc_path = 'data/baseline/02691156_2048_pc/1a04e3eab45ca15dd86060f189eb133.obj.npy'
pc_path = os.path.join(ROOT_DIR, pc_path)
print(f"Loading point cloud data from: {pc_path}")
point_cloud_data = np.load(pc_path)

# Determine the format and extract the coordinates to be analyzed.
if point_cloud_data.shape[1] == 4: # Occupancy format [x,y,z,occ]
    print("Detected occupancy format.")
    occupied_mask = point_cloud_data[:, 3] > 0.5
    points_to_analyze = point_cloud_data[occupied_mask][:, :3]
else: # Assume normal format [x,y,z,nx,ny,nz]
    print("Detected normal format.")
    points_to_analyze = point_cloud_data[:, :3]

num_points = len(points_to_analyze)
print(f"Ready to analyze {num_points} points.")

Loading point cloud data from: /home/lindert/TUM/ADL4CV/SemanticHyperDiffusion/data/baseline/02691156_2048_pc/1a04e3eab45ca15dd86060f189eb133.obj.npy
Detected normal format.
Ready to analyze 2048 points.


### Start Semantic Analysis

In [8]:
import plotly.graph_objects as go

# Load the raw point clouds without any normalization
raw_activations_coords = np.load(pc_path)[:, :3]
label_path = 'data/shapenetpart/PartAnnotation/02691156/points/1a04e3eab45ca15dd86060f189eb133.pts'
label_path = os.path.join(ROOT_DIR, label_path)

raw_labels_coords = np.loadtxt(label_path)

fig = go.Figure()

# Add the activation point cloud (source of truth) in blue
fig.add_trace(go.Scatter3d(
    x=raw_activations_coords[:, 0], y=raw_activations_coords[:, 1], z=raw_activations_coords[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.7),
    name='Activation Points (.npy)'
))

# Add the label point cloud in red
fig.add_trace(go.Scatter3d(
    x=raw_labels_coords[:, 0], y=raw_labels_coords[:, 1], z=raw_labels_coords[:, 2],
    mode='markers',
    marker=dict(size=2, color='red', opacity=0.7),
    name='Label Points (.pts)'
))

fig.update_layout(
    title='Visual Comparison of Raw Point Clouds (Before Alignment)',
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()

In [9]:
import numpy as np
import os
import json
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation

# Paths to the files
pc_npy_path = pc_path
pc_pts_path = label_path 

seg_path = 'data/shapenetpart/PartAnnotation/02691156/expert_verified/points_label/1a04e3eab45ca15dd86060f189eb133.seg'
seg_path = os.path.join(ROOT_DIR, seg_path)
metadata_path = '/data/shapenetpart/PartAnnotation/metadata.json'
metadata_path = os.path.join(ROOT_DIR, metadata_path)

category_name = "Airplane"

print("--- Manual Alignment and Semantic Data Loading ---")

coords_from_activations_raw = np.load(pc_npy_path)[:, :3]
coords_from_labels_raw = np.loadtxt(pc_pts_path)
original_part_labels = np.loadtxt(seg_path, dtype=int)
print(f"Loaded {len(coords_from_activations_raw)} points from activation source (.npy).")
print(f"Loaded {len(coords_from_labels_raw)} points from semantic label source (.pts).")

# Center and Normalize Both Point Clouds
def align_point_cloud(points):
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_abs_coord = np.max(np.abs(points_centered))
    points_scaled = points_centered / max_abs_coord
    return points_scaled

print("\nStep 1: Centering and scaling both point clouds...")
aligned_activations_coords = align_point_cloud(coords_from_activations_raw)
aligned_labels_coords = align_point_cloud(coords_from_labels_raw)
print("Centering and scaling complete.")

print("Step 2: Applying a 90-degree rotation around the Y-axis to the semantic point cloud...")
rotation = Rotation.from_euler('y', 90, degrees=True) # Using Y-axis as you discovered
manually_aligned_labels_coords = rotation.apply(aligned_labels_coords)
print("Rotation applied.")

print("Step 3: Finding closest points between the two aligned clouds...")
nn_search_tree = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(manually_aligned_labels_coords)
distances, closest_indices = nn_search_tree.kneighbors(aligned_activations_coords)

max_distance = np.max(distances)
print(f"Maximum distance between closest points after manual alignment: {max_distance:.2e}")

part_labels = None
if max_distance > 0.1: 
    print("\nERROR: The point clouds are still too different even after alignment.")
    print("Cannot reliably transfer semantic labels. Aborting semantic analysis.")
else:
    print("\nSUCCESS: Manual alignment successful! Point clouds are consistent enough. Remapping labels.")
    part_labels = original_part_labels[closest_indices.squeeze()]
    assert len(part_labels) == len(coords_from_activations_raw)

# --- 6. Load Semantic Names (only if successful) ---
if part_labels is not None:
    # We still load from metadata to be systematic, but we will override it.
    #with open(metadata_path, 'r') as f:
    #    metadata = json.load(f)

    print("\n--- Applying Manual Correction to Semantic Labels for Airplane ---")
    # Based on visual inspection, the labels in the .seg file for airplanes
    # correspond to: 1=Body, 2=Wing, 3=Tail, 4=Engine.
    # The JSON list is ['wing', 'body', ...], which creates the wrong order.
    # Therefore using manual part to label mapping
    part_label_to_name = {
        1: "Body",
        2: "Wing",
        3: "Tail",
        4: "Engine"
    }
    
    print("Corrected semantic part mapping:")
    for label_id, name in part_label_to_name.items():
        print(f"  Label {label_id}: {name}")
    
    unique_parts_in_scan = sorted(np.unique(part_labels))
    part_slider_options = [(name, label_id) for label_id, name in part_label_to_name.items() if label_id in unique_parts_in_scan]

--- Manual Alignment and Semantic Data Loading ---
Loaded 2048 points from activation source (.npy).
Loaded 2816 points from semantic label source (.pts).

Step 1: Centering and scaling both point clouds...
Centering and scaling complete.
Step 2: Applying a 90-degree rotation around the Y-axis to the semantic point cloud...
Rotation applied.
Step 3: Finding closest points between the two aligned clouds...
Maximum distance between closest points after manual alignment: 6.73e-02

SUCCESS: Manual alignment successful! Point clouds are consistent enough. Remapping labels.

--- Applying Manual Correction to Semantic Labels for Airplane ---
Corrected semantic part mapping:
  Label 1: Body
  Label 2: Wing
  Label 3: Tail
  Label 4: Engine


In [10]:
import plotly.graph_objects as go

if 'manually_aligned_labels_coords' in locals():
    print("--- Visualizing Alignment Result ---")
    fig = go.Figure()

    # Add the activation point cloud (our target) in blue
    fig.add_trace(go.Scatter3d(
        x=aligned_activations_coords[:, 0], y=aligned_activations_coords[:, 1], z=aligned_activations_coords[:, 2],
        mode='markers',
        marker=dict(size=2, color='blue', opacity=0.7),
        name='Activation Points (Aligned)'
    ))

    # Add the MANUALLY transformed label point cloud in red
    fig.add_trace(go.Scatter3d(
        x=manually_aligned_labels_coords[:, 0], y=manually_aligned_labels_coords[:, 1], z=manually_aligned_labels_coords[:, 2],
        mode='markers',
        marker=dict(size=2, color='red', opacity=0.7),
        name='Label Points (Manually Aligned)'
    ))

    fig.update_layout(
        title='Result of Manual Alignment: Blue and Red should overlap perfectly',
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()
else:
    print("Run the cell above to perform the alignment first.")

--- Visualizing Alignment Result ---


In [11]:
import plotly.graph_objects as go

# Ensure part_labels and part_label_to_name are loaded from the previous cell.
if 'part_labels' in locals() and 'part_label_to_name' in locals():
    print("--- Visualizing Current Semantic Labeling ---")
    
    fig = go.Figure()
    
    # Add a separate trace for each part so we can have a legend.
    for label_id, name in part_label_to_name.items():
        # Find all points that have this label
        points_for_this_part = points_to_analyze[part_labels == label_id]
        
        if len(points_for_this_part) > 0:
            fig.add_trace(go.Scatter3d(
                x=points_for_this_part[:, 0], 
                y=points_for_this_part[:, 1], 
                z=points_for_this_part[:, 2],
                mode='markers',
                marker=dict(size=2, opacity=0.8),
                name=f"{name} (Label {label_id})" # This will appear in the legend
            ))

    fig.update_layout(
        title='Full Point Cloud Colored by Current Semantic Labels',
        legend_title_text='Parts',
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

else:
    print("Please run the semantic data loading cell successfully first.")

--- Visualizing Current Semantic Labeling ---


In [12]:
from sklearn.cluster import KMeans

# Choose the number of clusters
n_clusters = 3

print(f"Performing K-Means clustering on {num_points} points to create {n_clusters} clusters...")

kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
# The fit_predict method runs the algorithm and returns an array of cluster labels
# cluster_labels[i] will be the cluster ID (from 0 to 49) for points_to_analyze[i]
cluster_labels = kmeans.fit_predict(points_to_analyze)

print("Clustering complete.")
print(f"Shape of cluster_labels array: {cluster_labels.shape}")
print(f"Example labels (first 10 points): {cluster_labels[:10]}")

Performing K-Means clustering on 2048 points to create 3 clusters...
Clustering complete.
Shape of cluster_labels array: (2048,)
Example labels (first 10 points): [1 0 1 1 1 0 0 0 0 1]


In [13]:
# This dictionary will store the averaged activations for each cluster. Same as 'activations', but the first dimension is n_clusters instead of num_points.
avg_cluster_activations = {
    name: torch.zeros(n_clusters, tensor.shape[1], device=tensor.device)
    for name, tensor in activations.items()
}

print("Calculating average activation vectors for each cluster...")

for c in range(n_clusters):
    # Find the indices of all points that belong to the current cluster
    point_indices_in_cluster = np.where(cluster_labels == c)[0]
    
    # Skip empty clusters
    if len(point_indices_in_cluster) == 0:
        continue
        
    # Calculate average activation per layer
    for layer_name, activation_tensor in activations.items():
        activations_for_cluster = activation_tensor[point_indices_in_cluster]
        
        # Compute the mean along the 'points' dimension (dim=0)
        avg_activations = torch.mean(activations_for_cluster, dim=0)
        
        avg_cluster_activations[layer_name][c] = avg_activations

print("Average cluster activations computed successfully.")
print(f"Shape of averaged activations for hidden_layer_0: {avg_cluster_activations['hidden_layer_0'].shape}")

Calculating average activation vectors for each cluster...
Average cluster activations computed successfully.
Shape of averaged activations for hidden_layer_0: torch.Size([3, 128])


In [14]:
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive_output
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, leaves_list
from IPython.display import display, clear_output

if 'activations' not in locals() or 'cluster_labels' not in locals() or 'part_labels' not in locals():
    print("Error: Please run all previous data loading and clustering cells first.")
else:
    post_activations = {name: torch.relu(tensor) for name, tensor in activations.items()}
    grouping_toggle = widgets.ToggleButtons(
        options=['K-Means', 'Semantic Parts', 'Whole Figure'],
        description='Group By:',
        value='K-Means'
    )
    
    initial_slider_options = [("Cluster " + str(i), i) for i in range(n_clusters)]
    group_slider = widgets.SelectionSlider(
        options=initial_slider_options,
        description='Select Group:',
        continuous_update=False
    )
    
    activation_toggle = widgets.ToggleButtons(
        options=['Pre-ReLU', 'Post-ReLU'],
        description='Activation Type:',
        value='Pre-ReLU'
    )
    sort_checkbox = widgets.Checkbox(
        value=True,
        description='Sort Neurons by Cluster'
    )

    def update_slider_options(change):
        """Observes the grouping_toggle and updates the group_slider's options and visibility."""
        with group_slider.hold_sync():
            if change['new'] == 'K-Means':
                group_slider.layout.visibility = 'visible'
                group_slider.options = [("Cluster " + str(i), i) for i in range(n_clusters)]
                group_slider.description = 'Cluster:'
                group_slider.value = 0
            elif change['new'] == 'Semantic Parts':
                group_slider.layout.visibility = 'visible'
                group_slider.options = part_slider_options
                group_slider.description = 'Part:'
                if part_slider_options:
                    group_slider.value = part_slider_options[0][1]
            else: # Whole Figure
                group_slider.layout.visibility = 'hidden'

    grouping_toggle.observe(update_slider_options, names='value')

    plot_output = widgets.Output()

    @plot_output.capture(clear_output=True, wait=True)
    def plot_correlation_heatmaps(grouping_method, selected_group_id, activation_type, sort_neurons):
        
        if grouping_method == 'Whole Figure':
            point_indices = np.arange(num_points)
            group_name = "Whole Figure"
        else:
            current_labels = cluster_labels if grouping_method == 'K-Means' else part_labels
            point_indices = np.where(current_labels == selected_group_id)[0]
            group_name = dict(group_slider.options).get(selected_group_id, f"ID {selected_group_id}")

        if len(point_indices) < 2:
            print(f"Not enough points in this group to compute correlation (found {len(point_indices)}).")
            return

        data_source = activations if activation_type == 'Pre-ReLU' else post_activations
        layer_names = sorted(data_source.keys())
        
        fig, axes = plt.subplots(1, len(layer_names), figsize=(18, 5.5))
        if len(layer_names) == 1: axes = [axes]

        fig.suptitle(f'Neuron Correlation for "{group_name}" ({activation_type})' + (' - Clustered' if sort_neurons else ''), fontsize=16)

        for i, layer_name in enumerate(layer_names):
            activations_for_group = data_source[layer_name][point_indices]
            correlation_matrix = torch.corrcoef(activations_for_group.T)
            correlation_matrix[torch.isnan(correlation_matrix)] = 0
            plot_data = correlation_matrix.cpu().numpy()
            
            if sort_neurons:
                dist_matrix = 1 - plot_data
                linked = linkage(dist_matrix, method='ward')
                sorted_indices = leaves_list(linked)
                plot_data = plot_data[sorted_indices, :][:, sorted_indices]

            ax = axes[i]
            sns.heatmap(
                plot_data, ax=ax, cmap='vlag', vmin=-1, vmax=1,
                xticklabels=False, yticklabels=False,
                cbar=(i == len(layer_names) - 1)
            )
            ax.set_title(f"Layer '{layer_name}'")
            ax.set_xlabel("Neuron Index" + (" (Sorted)" if sort_neurons else ""))
            if i == 0:
                ax.set_ylabel("Neuron Index" + (" (Sorted)" if sort_neurons else ""))

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    # Link widgets and display
    interactive_plot = interactive_output(plot_correlation_heatmaps, {
        'grouping_method': grouping_toggle,
        'selected_group_id': group_slider,
        'activation_type': activation_toggle,
        'sort_neurons': sort_checkbox
    })
    
    controls = VBox([
        HBox([grouping_toggle, activation_toggle]),
        HBox([group_slider, sort_checkbox])
    ])
    
    display(VBox([controls, plot_output]))
    
    # Manually initialize the UI
    update_slider_options({'new': grouping_toggle.value})
    plot_correlation_heatmaps(
        grouping_method=grouping_toggle.value,
        selected_group_id=group_slider.value,
        activation_type=activation_toggle.value,
        sort_neurons=sort_checkbox.value
    )

In [19]:
import ipywidgets as widgets
from ipywidgets import VBox, HBox
import torch
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from scipy.stats import pearsonr

# Ensure all necessary data from previous cells is available.
if 'activations' not in locals() or 'cluster_labels' not in locals() or 'part_labels' not in locals():
    print("Error: Please run all previous data loading and pre-computation cells first.")
else:
    # --- 1. Create all UI components ---
    available_layers = sorted(activations.keys())
    num_neurons = activations[available_layers[0]].shape[1]

    # A. Interactive Controls
    grouping_toggle = widgets.ToggleButtons(options=['K-Means', 'Semantic Parts', 'Whole Figure'], description='Group By:')
    initial_slider_options = [("Cluster " + str(i), i) for i in range(n_clusters)]
    group_slider = widgets.SelectionSlider(options=initial_slider_options, description='Group:', continuous_update=False)
    layer_selector = widgets.Dropdown(options=available_layers, value=available_layers[0], description='Layer:')
    activation_toggle = widgets.ToggleButtons(options=['Pre-ReLU', 'Post-ReLU'], description='Activation:')
    log_scale_checkbox = widgets.Checkbox(value=True, description='Log Scale Y-Axis')
    center_tendency_toggle = widgets.ToggleButtons(options=['Mean', 'Median'], description='X-Axis:')
    dispersion_toggle = widgets.ToggleButtons(options=['Variance', 'MAD'], description='Y-Axis:')

    # B. Plot Canvases - Initialized with placeholder traces
    cluster_plot_widget = go.FigureWidget(data=[
        go.Scatter3d(x=points_to_analyze[:, 0], y=points_to_analyze[:, 1], z=points_to_analyze[:, 2],
                      mode='markers', marker=dict(size=2, color='lightgrey'))
    ])
    scatter_widget = go.FigureWidget(data=[go.Scatter(x=[], y=[], mode='markers')])
    heatmap_widget = go.FigureWidget(data=[go.Heatmap(z=np.zeros((4, num_neurons)))])

    # --- 2. Define functions to manage UI state and updates ---
    def update_slider_options(change):
        with group_slider.hold_sync():
            if change['new'] == 'K-Means':
                group_slider.layout.visibility = 'visible'
                group_slider.options = [("Cluster " + str(i), i) for i in range(n_clusters)]
                group_slider.description = 'Cluster:'
                group_slider.value = 0
            elif change['new'] == 'Semantic Parts':
                group_slider.layout.visibility = 'visible'
                group_slider.options = part_slider_options
                group_slider.description = 'Part:'
                if part_slider_options:
                    group_slider.value = part_slider_options[0][1]
            else: # Whole Figure
                group_slider.layout.visibility = 'hidden'
    
    grouping_toggle.observe(update_slider_options, names='value')

    def handle_master_update(change):
        grouping_method = grouping_toggle.value
        selected_id = group_slider.value
        
        # --- Determine point indices and display name ---
        if grouping_method == 'Whole Figure':
            point_indices = np.arange(num_points)
            display_name = 'Whole Figure'
        else:
            current_labels = cluster_labels if grouping_method == 'K-Means' else part_labels
            point_indices = np.where(current_labels == selected_id)[0]
            display_name = f'Cluster #{selected_id}' if grouping_method == 'K-Means' else f'Part: {dict(group_slider.options).get(selected_id, "Unknown")}'

        # --- Update 3D plot (in-place) ---
        colors = np.array(['lightgrey'] * num_points)
        if grouping_method == 'Whole Figure':
            colors[:] = 'red'
        else:
            colors[point_indices] = 'red'
            
        with cluster_plot_widget.batch_update():
            # CORRECT: Modify the 'marker.color' property of the existing trace
            cluster_plot_widget.data[0].marker.color = colors
            cluster_plot_widget.update_layout(title_text=display_name, title_x=0.5, margin=dict(l=0,r=0,b=0,t=40),
                                              scene_camera_eye=dict(x=1.5, y=1.5, z=1.5), showlegend=False)
        
        # --- Update 2D plots (Scatter and Heatmap) ---
        if len(point_indices) < 2:
            for widget in [scatter_widget, heatmap_widget]:
                with widget.batch_update(): widget.data[0].visible = False; widget.update_layout(title_text="Not enough points in group")
            return
        
        selected_layer = layer_selector.value; activation_type = activation_toggle.value
        use_log_scale = log_scale_checkbox.value
        center_stat_key = 'median' if center_tendency_toggle.value == 'Median' else 'mean'
        dispersion_stat_key = 'mad' if dispersion_toggle.value == 'MAD' else 'var'

        pre_acts = activations[selected_layer][point_indices].cpu()
        act_source = torch.relu(pre_acts) if activation_type == 'Post-ReLU' else pre_acts
        
        mean_vec=torch.mean(act_source, dim=0).numpy(); median_vec=torch.median(act_source, dim=0).values.numpy()
        var_vec=torch.var(act_source, dim=0).numpy(); mad_vec=(torch.median(torch.abs(act_source - torch.from_numpy(median_vec)), dim=0).values * 1.4826).numpy()
        x_vec=median_vec if center_stat_key == 'median' else mean_vec
        y_vec_scatter=mad_vec if dispersion_stat_key == 'mad' else var_vec
        plot_y_vec_scatter=np.log1p(y_vec_scatter) if use_log_scale else y_vec_scatter
        correlation, _ = pearsonr(mean_vec, var_vec + 1e-9)
        
        with scatter_widget.batch_update():
            scatter_widget.data[0].visible = True; scatter_widget.data[0].x = x_vec; scatter_widget.data[0].y = plot_y_vec_scatter
            scatter_widget.data[0].hovertext=[f'Neuron {i}' for i in range(len(x_vec))]; scatter_widget.data[0].hoverinfo='x+y+text'
            scatter_widget.update_layout(title_text=f"Correlation Plot<br>Mean/Var Corr: {correlation:.3f}", title_x=0.5,
                                         xaxis_title=f"{center_stat_key.capitalize()} {activation_type}",
                                         yaxis_title=f"{'Log ' if use_log_scale else ''}{dispersion_toggle.value}",
                                         margin=dict(l=20, r=20, b=20, t=60))
            
        plot_var_vec_hm=np.log1p(var_vec) if use_log_scale else var_vec; plot_mad_vec_hm=np.log1p(mad_vec) if use_log_scale else mad_vec
        heatmap_z = np.vstack([mean_vec, median_vec, plot_var_vec_hm, plot_mad_vec_hm])
        heatmap_y = ["Mean", "Median", f"{'Log ' if use_log_scale else ''}Var", f"{'Log ' if use_log_scale else ''}MAD"]
        
        with heatmap_widget.batch_update():
            heatmap_widget.data[0].visible = True; heatmap_widget.data[0].z = heatmap_z; heatmap_widget.data[0].y = heatmap_y
            heatmap_widget.data[0].colorscale='RdBu' if activation_type == 'Pre-ReLU' else 'Viridis'
            heatmap_widget.data[0].zmid=0 if activation_type == 'Pre-ReLU' else None
            heatmap_widget.update_layout(title_text=f'Activation Signature for "{selected_layer}"', title_x=0.5,
                                         xaxis_title="Neuron Index", yaxis_title="Statistic", margin=dict(l=20,r=20,b=20,t=60))

    # --- 3. Link widgets and display ---
    widgets_to_observe = [grouping_toggle, group_slider, layer_selector, activation_toggle, log_scale_checkbox, center_tendency_toggle, dispersion_toggle]
    for widget in widgets_to_observe:
        widget.observe(handle_master_update, names='value')

    controls1 = HBox([grouping_toggle, layer_selector])
    controls2 = HBox([group_slider, activation_toggle])
    controls3 = HBox([center_tendency_toggle, dispersion_toggle, log_scale_checkbox])
    controls = VBox([controls1, controls2, controls3])
    
    plots = HBox([cluster_plot_widget, scatter_widget, heatmap_widget])
    
    display(VBox([controls, plots]))
    
    update_slider_options({'new': grouping_toggle.value})
    handle_master_update(None)

In [18]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output

if 'activations' not in locals() or 'cluster_labels' not in locals() or 'part_labels' not in locals():
    print("Error: Please run all previous data loading and pre-computation cells first.")
else:
    def calculate_new_metrics(activations_tensor, point_indices_in_group, all_point_indices):
        """
        Calculates a dictionary of metrics for a given group of activations.
        """
        if len(point_indices_in_group) < 2:
            return {}

        group_activations_pre = activations_tensor[point_indices_in_group]
        group_activations_post = torch.relu(group_activations_pre)
        
        # Sparsity: what percentage of neuron activations are > 0 for this group?
        is_neuron_active = group_activations_post > 0
        sparsity = is_neuron_active.float().mean().item() * 100
        
        # Find Top 5 Neurons for this Group
        avg_post_relu_activations = torch.mean(group_activations_post, dim=0)
        topk_k = min(5, group_activations_pre.shape[1])
        if topk_k == 0: return {}
        top_neurons_indices = torch.topk(avg_post_relu_activations, k=topk_k).indices
        
        avg_top5_mean_act = avg_post_relu_activations[top_neurons_indices].mean().item()

        MAD_SCALE_FACTOR = 1.4826
        top5_activations_pre = group_activations_pre[:, top_neurons_indices]
        top5_median = torch.median(top5_activations_pre, dim=0).values
        top5_mad_values = torch.median(torch.abs(top5_activations_pre - top5_median), dim=0).values * MAD_SCALE_FACTOR
        avg_top5_mad = top5_mad_values.mean().item()

        outside_indices = np.setdiff1d(all_point_indices, point_indices_in_group)
        specialized_neuron_count = 0
        if len(outside_indices) > 0:
            mean_on_rest_post = torch.mean(torch.relu(activations_tensor[outside_indices]), dim=0)
            is_specialized = avg_post_relu_activations > (2 * mean_on_rest_post)
            specialized_neuron_count = torch.sum(is_specialized).item()
        
        return {
            "Sparsity (%)": sparsity,
            "Avg Top-5 Mean Act": avg_top5_mean_act,
            "Avg Top-5 MAD": avg_top5_mad,
            "Specialized Neurons": specialized_neuron_count,
            "Top 5 Neuron IDs": top_neurons_indices.cpu().numpy()
        }

    layer_selector = widgets.Dropdown(options=available_layers, value=available_layers[2], description='Layer:')
    output_area = widgets.Output()

    def handle_analysis_update(change):
        with output_area:
            clear_output(wait=True)
            selected_layer = layer_selector.value
            results_list = []
            
            for grouping_method in ['K-Means', 'Semantic Parts', 'Whole Figure']:
                if grouping_method == 'K-Means':
                    labels, group_ids, group_names = cluster_labels, np.unique(cluster_labels), {i: f"Cluster {i}" for i in np.unique(cluster_labels)}
                elif grouping_method == 'Semantic Parts':
                    labels, group_ids, group_names = part_labels, np.unique(part_labels), part_label_to_name
                else:
                    labels, group_ids, group_names = np.zeros(num_points, dtype=int), [0], {0: "Whole Figure"}

                all_point_indices = np.arange(activations[selected_layer].shape[0])
                layer_results = {}
                for group_id in group_ids:
                    point_indices = np.where(labels == group_id)[0]
                    metrics = calculate_new_metrics(activations[selected_layer], point_indices, all_point_indices)
                    layer_results[group_id] = metrics
                
                if grouping_method != 'Whole Figure':
                    for group_id_i in group_ids:
                        if not layer_results.get(group_id_i): continue
                        top_neurons_i = set(layer_results[group_id_i]["Top 5 Neuron IDs"])
                        jaccard_scores = []
                        for group_id_j in group_ids:
                            if group_id_i == group_id_j or not layer_results.get(group_id_j): continue
                            top_neurons_j = set(layer_results[group_id_j]["Top 5 Neuron IDs"])
                            intersection, union = len(top_neurons_i.intersection(top_neurons_j)), len(top_neurons_i.union(top_neurons_j))
                            jaccard_scores.append(intersection / union if union > 0 else 0)
                        layer_results[group_id_i]['Disentanglement (Avg Jaccard)'] = np.mean(jaccard_scores) if jaccard_scores else 0
                else:
                    if layer_results.get(0):
                        layer_results[0]['Disentanglement (Avg Jaccard)'] = np.nan
                
                for group_id in group_ids:
                    if not layer_results.get(group_id): continue
                    res = {"Grouping": grouping_method, "Group Name": group_names.get(group_id, f"ID {group_id}")}
                    res.update(layer_results[group_id])
                    del res["Top 5 Neuron IDs"]
                    results_list.append(res)
            
            df = pd.DataFrame(results_list)

            avg_kmeans = df[df['Grouping'] == 'K-Means'].mean(numeric_only=True)
            avg_semantic = df[df['Grouping'] == 'Semantic Parts'].mean(numeric_only=True)
            avg_kmeans['Grouping'], avg_kmeans['Group Name'] = "Summary", "K-Means (Avg)"
            avg_semantic['Grouping'], avg_semantic['Group Name'] = "Summary", "Semantic Parts (Avg)"
            df = pd.concat([df, avg_kmeans.to_frame().T, avg_semantic.to_frame().T], ignore_index=True)
            df = df.set_index(["Grouping", "Group Name"])
            
            style_cols_green = ['Avg Top-5 Mean Act', 'Specialized Neurons']
            style_cols_red = ['Avg Top-5 MAD', 'Disentanglement (Avg Jaccard)', 'Sparsity (%)']
            
            styled_df = df.style.background_gradient(cmap='viridis', subset=style_cols_green, axis=0) \
                                 .background_gradient(cmap='viridis_r', subset=style_cols_red, axis=0) \
                                 .format("{:.2f}", na_rep="-")

            print(f"--- Quantitative Analysis for Layer: '{selected_layer}' ---")
            display(styled_df)
            
    layer_selector.observe(handle_analysis_update, names='value')
    display(VBox([layer_selector, output_area]))
    handle_analysis_update(None)

In [20]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output

# Load the pre-computed results created by quant_act_analysis.py
csv_path = 'quantitative_analysis_results.csv'
if not os.path.exists(csv_path):
    print(f"Error: CSV file not found at '{csv_path}'. Please run the 'run_quantitative_analysis.py' script first.")
else:
    df_full = pd.read_csv(csv_path)
    print(f"Successfully loaded {len(df_full)} rows from {csv_path}")

    #interactive widgets
    available_layers = sorted(df_full['Layer'].unique())
    layer_selector = widgets.Dropdown(options=available_layers, value=available_layers[0], description='Layer:')
    output_area = widgets.Output()

    def handle_summary_update(change):
        with output_area:
            clear_output(wait=True)
            selected_layer = layer_selector.value

            # Filter by selected layer
            df_layer = df_full[df_full['Layer'] == selected_layer].copy()

            df_kmeans = df_layer[df_layer['Grouping'] == 'K-Means']
            df_semantic = df_layer[df_layer['Grouping'] == 'Semantic Parts']
            df_whole = df_layer[df_layer['Grouping'] == 'Whole Figure']

            # Create group averages across shapes
            summary_kmeans = df_kmeans.groupby('Group Name').mean(numeric_only=True)
            summary_semantic = df_semantic.groupby('Group Name').mean(numeric_only=True)
            summary_whole = df_whole.groupby('Group Name').mean(numeric_only=True)

            # Calculate the overall averages and create summary rows
            avg_kmeans_row = summary_kmeans.mean()
            avg_semantic_row = summary_semantic.mean()
            
            # Create a list of tuples for the new hierarchical index
            final_index = []
            final_data = []

            for name, row in summary_kmeans.iterrows():
                final_index.append(('K-Means', name))
                final_data.append(row)

            for name, row in summary_semantic.iterrows():
                final_index.append(('Semantic Parts', name))
                final_data.append(row)
            
            for name, row in summary_whole.iterrows():
                final_index.append(('Whole Figure', name))
                final_data.append(row)

            # Summary rows
            final_index.append(('Summary', 'K-Means (Avg)'))
            final_data.append(avg_kmeans_row)
            final_index.append(('Summary', 'Semantic Parts (Avg)'))
            final_data.append(avg_semantic_row)
            
            final_df = pd.DataFrame(final_data, index=pd.MultiIndex.from_tuples(final_index, names=["Grouping", "Group Name"]))

            style_cols_green = ['Avg Top-5 Mean Act', 'Specialized Neurons']
            style_cols_red = ['Avg Top-5 MAD', 'Disentanglement (Avg Jaccard)', 'Sparsity (%)']
            
            valid_style_cols_green = [col for col in style_cols_green if col in final_df.columns]
            valid_style_cols_red = [col for col in style_cols_red if col in final_df.columns]
            
            styled_df = final_df.style.background_gradient(cmap='viridis', subset=valid_style_cols_green, axis=0) \
                                      .background_gradient(cmap='viridis_r', subset=valid_style_cols_red, axis=0) \
                                      .format("{:.2f}", na_rep="-")

            print(f"--- Average Metrics Across All Shapes for Layer: '{selected_layer}' ---")
            display(styled_df)
            
    layer_selector.observe(handle_summary_update, names='value')
    display(VBox([layer_selector, output_area]))
    handle_summary_update(None)

Error: CSV file not found at 'quantitative_analysis_results.csv'. Please run the 'run_quantitative_analysis.py' script first.
